In [1]:
# CARREGAR DADOS PRÉ-PROCESSADOS

import joblib
import numpy as np

print("=" * 80)
print("CARREGANDO DADOS PRÉ-PROCESSADOS")
print("=" * 80)

# Carregar dados já tratados
data_package = joblib.load("diabetes_model/preprocessed_data.joblib")

X_train = data_package['X_train']
X_test = data_package['X_test']
y_train = data_package['y_train']
y_test = data_package['y_test']

print(f"✅ Dados carregados:")
print(f"   X_train: {X_train.shape}")
print(f"   X_test: {X_test.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   y_test: {y_test.shape}")

print("\n✅ PRONTO PARA ALGORITMO GENÉTICO!")

CARREGANDO DADOS PRÉ-PROCESSADOS
✅ Dados carregados:
   X_train: (95996, 18)
   X_test: (24000, 18)
   y_train: (95996,)
   y_test: (24000,)

✅ PRONTO PARA ALGORITMO GENÉTICO!


In [2]:
# CONFIGURAÇÃO RÁPIDA PARA TESTE - ALGORITMO GENÉTICO OTIMIZADO

import random
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.linear_model import LogisticRegression
from deap import base, creator, tools, algorithms

print("=" * 80)
print("CONFIGURAÇÃO OTIMIZADA - TESTE RÁPIDO")
print("=" * 80)

# 1. FUNÇÃO DE CRIAÇÃO (já cria com tipos corretos)
def create_individual():
    """Cria indivíduo com tipos corretos desde o início"""
    return [
        random.uniform(0.01, 100.0),    # Gene 0: C (float)
        random.randint(0, 2),           # Gene 1: penalty_type (int)
        random.randint(0, 3),           # Gene 2: solver_type (int)
        random.randint(0, 1),           # Gene 3: class_weight_type (int)
        random.randint(500, 3000)       # Gene 4: max_iter (int)
    ]

# 2. FUNÇÃO DE AVALIAÇÃO (sem necessidade de clamping!)
def evaluate_individual(individual):
    """Avalia indivíduo - sem clamping necessário"""
    C, penalty_type, solver_type, class_weight_type, max_iter = individual
    
    # Mapear para valores reais
    penalty_map = {0: 'l2', 1: 'l1', 2: 'elasticnet'}
    solver_map = {0: 'lbfgs', 1: 'liblinear', 2: 'saga', 3: 'newton-cg'}
    class_weight_map = {0: None, 1: 'balanced'}
    
    penalty = penalty_map[penalty_type]
    solver = solver_map[solver_type]
    class_weight = class_weight_map[class_weight_type]
    
    # Validar combinações inválidas
    if penalty == 'l1' and solver not in ['liblinear', 'saga']:
        return (-1000,)
    if penalty == 'elasticnet' and solver != 'saga':
        return (-1000,)
    
    try:
        model = LogisticRegression(
            C=C,
            penalty=penalty,
            solver=solver,
            class_weight=class_weight,
            max_iter=int(max_iter),
            random_state=42,
            n_jobs=-1
        )
        
        # TESTE RÁPIDO: apenas 2 folds (33% mais rápido)
        scorer = make_scorer(f1_score)
        scores = cross_val_score(model, X_train, y_train, cv=2, scoring=scorer, n_jobs=-1)
        
        return (scores.mean(),)
    
    except:
        return (-1000,)

# 3. CROSSOVER HÍBRIDO (preserva tipos inteiros)
def hybrid_crossover(ind1, ind2):
    """Crossover que preserva tipos: blend para float, uniform para inteiros"""
    child1, child2 = creator.Individual(ind1[:]), creator.Individual(ind2[:])
    
    # Gene 0 (C - float): Blend crossover
    alpha = 0.5
    gamma = (1 + 2 * alpha) * random.random() - alpha
    child1[0] = (1 - gamma) * ind1[0] + gamma * ind2[0]
    child2[0] = (1 - gamma) * ind2[0] + gamma * ind1[0]
    
    # Limitar C entre 0.01 e 100.0
    child1[0] = max(0.01, min(100.0, child1[0]))
    child2[0] = max(0.01, min(100.0, child2[0]))
    
    # Genes 1-4 (inteiros): Uniform crossover (50% de chance de trocar)
    for i in range(1, 5):
        if random.random() < 0.5:
            child1[i], child2[i] = ind2[i], ind1[i]
    
    return child1, child2

# 4. MUTAÇÃO (já otimizada para tipos mistos)
def hybrid_mutate(individual):
    """Mutação que respeita tipos"""
    # Gene 0 (C - float): multiplicação com fator aleatório
    if random.random() < 0.3:
        individual[0] *= random.uniform(0.5, 2.0)
        individual[0] = max(0.01, min(100.0, individual[0]))
    
    # Genes 1-4 (inteiros): nova amostragem aleatória
    if random.random() < 0.3:
        individual[1] = random.randint(0, 2)
    if random.random() < 0.3:
        individual[2] = random.randint(0, 3)
    if random.random() < 0.3:
        individual[3] = random.randint(0, 1)
    if random.random() < 0.3:
        individual[4] = random.randint(500, 3000)
    
    return individual,

# 5. CONFIGURAR DEAP
# Limpar definições anteriores
if hasattr(creator, "FitnessMax"):
    del creator.FitnessMax
if hasattr(creator, "Individual"):
    del creator.Individual

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_individual)
toolbox.register("mate", hybrid_crossover)      # ← Crossover híbrido!
toolbox.register("mutate", hybrid_mutate)       # ← Mutação híbrida!
toolbox.register("select", tools.selTournament, tournsize=3)

print("\n✅ Configuração completa!")
print("   • create_individual: Tipos corretos ✓")
print("   • evaluate_individual: Sem clamping ✓")
print("   • hybrid_crossover: Preserva inteiros ✓")
print("   • hybrid_mutate: Otimizado ✓")

CONFIGURAÇÃO OTIMIZADA - TESTE RÁPIDO

✅ Configuração completa!
   • create_individual: Tipos corretos ✓
   • evaluate_individual: Sem clamping ✓
   • hybrid_crossover: Preserva inteiros ✓
   • hybrid_mutate: Otimizado ✓


In [3]:
# EXECUÇÃO RÁPIDA - TESTE (5-10 minutos)

print("\n" + "=" * 80)
print("EXECUTANDO TESTE RÁPIDO")
print("=" * 80)

random.seed(42)
np.random.seed(42)

# CONFIGURAÇÃO RÁPIDA
POPULATION_SIZE = 15   # Reduzido de 30 (50% mais rápido)
NGEN = 10              # Reduzido de 20 (50% mais rápido)
CXPB = 0.7
MUTPB = 0.3

print(f"\n📊 Parâmetros do teste rápido:")
print(f"   • População: {POPULATION_SIZE} (reduzida)")
print(f"   • Gerações: {NGEN} (reduzida)")
print(f"   • Cross-validation: 2 folds (reduzida)")
print(f"   • Crossover: {CXPB}")
print(f"   • Mutação: {MUTPB}")
print(f"\n⏱️  Tempo estimado: 5-10 minutos")

# Criar população
population = toolbox.population(n=POPULATION_SIZE)

# Estatísticas
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Hall of Fame
hof = tools.HallOfFame(5)

print("\n🚀 Iniciando evolução...")
print("-" * 80)

# EXECUTAR
population, logbook = algorithms.eaSimple(
    population, 
    toolbox, 
    cxpb=CXPB, 
    mutpb=MUTPB, 
    ngen=NGEN,
    stats=stats, 
    halloffame=hof, 
    verbose=True
)

print("\n" + "=" * 80)
print("✅ TESTE CONCLUÍDO!")
print("=" * 80)


EXECUTANDO TESTE RÁPIDO

📊 Parâmetros do teste rápido:
   • População: 15 (reduzida)
   • Gerações: 10 (reduzida)
   • Cross-validation: 2 folds (reduzida)
   • Crossover: 0.7
   • Mutação: 0.3

⏱️  Tempo estimado: 5-10 minutos

🚀 Iniciando evolução...
--------------------------------------------------------------------------------


/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib

gen	nevals	avg     	std    	min  	max     
0  	15    	-199.287	400.356	-1000	0.891196


/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be remo

1  	12    	0.887258	0.0091233	0.863998	0.890967


...

8  	13    	0.89105 	7.5871e-05 	0.890815	0.891168


/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib

9  	7     	0.891094	7.65238e-05	0.890951	0.891168


/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be remo

10 	12    	0.891152	4.25207e-05	0.891032	0.891168

✅ TESTE CONCLUÍDO!


In [7]:
# CALCULAR MÉTRICAS DO MODELO OTIMIZADO

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Calculando métricas do modelo otimizado...")

# Pegar melhor indivíduo do Hall of Fame
best_individual = hof[0]
C_best, penalty_type, solver_type, class_weight_type, max_iter_best = best_individual

# Mapear para valores reais
penalty_map = {0: 'l2', 1: 'l1', 2: 'elasticnet'}
solver_map = {0: 'lbfgs', 1: 'liblinear', 2: 'saga', 3: 'newton-cg'}
class_weight_map = {0: None, 1: 'balanced'}

penalty_best = penalty_map[penalty_type]
solver_best = solver_map[solver_type]
class_weight_best = class_weight_map[class_weight_type]

# Treinar modelo final com melhor configuração
best_model = LogisticRegression(
    C=C_best,
    penalty=penalty_best,
    solver=solver_best,
    class_weight=class_weight_best,
    max_iter=int(max_iter_best),
    random_state=42
)

best_model.fit(X_train, y_train)
y_pred_ga = best_model.predict(X_test)

# Calcular métricas
accuracy_ga = accuracy_score(y_test, y_pred_ga) * 100
precision_ga = precision_score(y_test, y_pred_ga) * 100
recall_ga = recall_score(y_test, y_pred_ga) * 100
f1_ga = f1_score(y_test, y_pred_ga) * 100

print(f"✅ Métricas calculadas:")
print(f"   Accuracy: {accuracy_ga:.2f}%")
print(f"   Precision: {precision_ga:.2f}%")
print(f"   Recall: {recall_ga:.2f}%")
print(f"   F1 Score: {f1_ga:.2f}%")

Calculando métricas do modelo otimizado...


/Users/pedromarquardt/dev/estudos/tech-challenger/ml-diabetes-detection/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


✅ Métricas calculadas:
   Accuracy: 89.46%
   Precision: 90.81%
   Recall: 87.82%
   F1 Score: 89.29%


In [8]:
# COMPARAÇÃO: ORIGINAL vs OTIMIZADO

import pandas as pd

print("=" * 80)
print("COMPARAÇÃO: MODELO ORIGINAL vs OTIMIZADO")
print("=" * 80)

# Carregar modelo original
model_package = joblib.load("diabetes_model/diabetes_model.joblib")
original_model = model_package['model']
y_pred_original = original_model.predict(X_test)

# Métricas do original
accuracy_original = accuracy_score(y_test, y_pred_original) * 100
precision_original = precision_score(y_test, y_pred_original) * 100
recall_original = recall_score(y_test, y_pred_original) * 100
f1_original = f1_score(y_test, y_pred_original) * 100

# Comparação
comparison_df = pd.DataFrame({
    'Métrica': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Original': [accuracy_original, precision_original, recall_original, f1_original],
    'Otimizado (AG)': [accuracy_ga, precision_ga, recall_ga, f1_ga],
    'Melhoria': [
        accuracy_ga - accuracy_original,
        precision_ga - precision_original,
        recall_ga - recall_original,
        f1_ga - f1_original
    ]
})

print("\n")
print(comparison_df.round(2).to_string(index=False))

# Análise
improvement = f1_ga - f1_original

print(f"\n{'='*80}")
print("ANÁLISE DA MELHORIA")
print(f"{'='*80}")

print(f"\n📊 F1 Score:")
print(f"   Original: {f1_original:.2f}%")
print(f"   Otimizado: {f1_ga:.2f}%")
print(f"   Melhoria: {improvement:+.2f} pontos percentuais")
print(f"   Melhoria relativa: {(improvement/f1_original*100):+.2f}%")

if improvement > 0.5:
    print("\n✅ EXCELENTE! Melhoria significativa!")
    print("   Recomendação: Salvar e usar este modelo")
elif improvement > 0:
    print("\n✅ BOM! Pequena melhoria")
    print("   Recomendação: Considere rodar versão completa (20 gerações, pop=30)")
else:
    print("\n⚠️  Sem melhoria ou piorou")
    print("   Recomendação: Manter modelo original")

COMPARAÇÃO: MODELO ORIGINAL vs OTIMIZADO


  Métrica  Original  Otimizado (AG)  Melhoria
 Accuracy     89.46           89.46      0.00
Precision     90.85           90.81     -0.05
   Recall     87.76           87.82      0.06
 F1 Score     89.28           89.29      0.01

ANÁLISE DA MELHORIA

📊 F1 Score:
   Original: 89.28%
   Otimizado: 89.29%
   Melhoria: +0.01 pontos percentuais
   Melhoria relativa: +0.01%

✅ BOM! Pequena melhoria
   Recomendação: Considere rodar versão completa (20 gerações, pop=30)
